---
title: Table of contents
jupyter:
  jupytext:
    text_representation:
      extension: .qmd
      format_name: quarto
      format_version: '1.0'
      jupytext_version: 1.16.2
  kernelspec:
    display_name: Python 3 (ipykernel)
    language: python
    name: python3
---

Exploration of whether folium can be used for all my use cases?
1. [OneMap API calls](#OneMap)
1. [Routes](#Routes)
1. [Regions](#Regions)
1. [Spatial Query](#SpatialQuery)

### Things to think about
1. Collect more layers that allow interactions across different amenities
1. Establishing of walking routes from home to different amenities


In [162]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [163]:
import os
import time
import folium
import polyline
import random
import requests
import numpy as np
import pandas as pd
from glob import glob
import geopy.distance
from tqdm.notebook import tqdm
from html2image import Html2Image
import folium.plugins as plugins
from folium.plugins import MarkerCluster

base_folder = os.getcwd()
os.chdir(os.environ.get('cred_folder'))
import onemap_fun as om
os.chdir(base_folder)

### OneMAP

In [4]:
latlong_filename = 'home_loc_tests.csv'

if glob(latlong_filename):
    print("Loading in local file that already exists!")
    final_df = pd.read_csv(latlong_filename)

else:
    df = pd.read_csv("data/local_df.csv")
    df['api_request'] = df['block'] + " " + df['street_name']
    df.api_request[0]

    sub_df = df.head(50)
    sub_df.shape

    onemap_results = list()
    for kw in tqdm(sub_df.api_request):
        onemap_results.append(om.loc_best_match(kw))

    sub_df['om_api'] = onemap_results

    expanded_om_results = sub_df['om_api'].apply(pd.Series)
    sub_df = pd.concat([sub_df, expanded_om_results], axis=1)
    final_df = sub_df[['town', 'flat', 'area', 'price', 'api_request', 
                       'SEARCHVAL', 'BLK_NO', 'ADDRESS', 'POSTAL', 'LATITUDE', 'LONGITUDE', 'score', 'search_keyword']]
    
    final_df.to_csv("data/home_loc_tests.csv", index=False)

Loading in local file that already exists!


In [5]:
final_df.shape

(50, 13)

#### Add mrt file 

In [6]:
mrt_df = pd.read_csv("mrt_locations.csv")
mrt_df.head()

,code_string,code_num,BUILDING,POSTAL,X,Y,LATITUDE,LONGITUDE
0,TE,1,WOODLANDS NORTH,737668,22699.991245,47770.394573,1.448292,103.785693
1,TE,2,WOODLANDS,737736,22949.919132,46417.522335,1.436058,103.787939
2,TE,3,WOODLANDS SOUTH,737741,23542.554570,45459.758822,1.427396,103.793264
3,NS,1,JURONG EAST,609690,17869.057052,35038.968870,1.333153,103.742286
4,NS,2,BUKIT BATOK,659958,18679.322319,36794.926021,1.349033,103.749566


### Functions for Mapping tasks 

In [8]:
def table_select_from_pt(df: pd.DataFrame, loc_ll: tuple, select = True, select_radius = 1000) -> pd.DataFrame:
    """ Takes a Tuple Lat-Long input, a set of tables with Lat-Longs, and filters for include or exclude"""
    
    df_lat, df_long = df['LATITUDE'].tolist(), df['LONGITUDE'].tolist()
    df['dist'] = [geopy.distance.geodesic((float(i), float(j)), loc_ll).m for i,j in zip(df_lat, df_long)]
    if select: 
        selected_df = df[df['dist'] <= select_radius].reset_index(drop=True)
    else: 
        selected_df = df[df['dist'] > select_radius].reset_index(drop=True)
    
    return selected_df

In [87]:
def create_route(start_latlong: tuple, end_latlong: tuple, route_type: str = 'walk') -> list:
    """ Creates """
    
    # Route creation
    route_output = om.make_route_api(start_latlong, end_latlong, route_type=route_type)
    total_d = om.get_total_route_dist(route_output)

    # Processing walking route
    route_g = route_output.json().get('route_geometry')
    l = polyline.decode(route_g)
    routing = pd.concat([pd.Series(l), pd.Series(l)], axis=1)
    routing[1] = routing[1].shift(-1)
    route_list = [(i,j) for i,j in zip(routing[0], routing[1]) if j != None]
    
    return route_list, total_d

In [117]:
def generate_color(): 
    # Generate random values for red, green, and blue 
    r = random.randint(0, 255) 
    g = random.randint(0, 255) 
    b = random.randint(0, 255) 
    
    return f"rgb({r}, {g}, {b})"
 
print(generate_color())

rgb(195, 48, 2)


### Simple Folium Map
1. This is a simple map with different location points

In [9]:
make_cluster = False
# Define coordinates of where we want to center our map
sg_coords = [final_df.loc[0]['LATITUDE'], final_df.loc[0]['LONGITUDE']]

# Create Map
hm = folium.Map(location = sg_coords, titles="OpenStreetMap", zoom_start=16)

if make_cluster:
    marker_cluster = MarkerCluster().add_to(hm)
    for i in range(final_df.shape[0]):
        folium.Marker([final_df.loc[i]['LATITUDE'], final_df.loc[i]['LONGITUDE']],
                          popup = final_df.loc[i]['ADDRESS'],
                          icon=folium.Icon(color="red"),
                         ).add_to(marker_cluster)
else:
    for i in range(final_df.shape[0]):
        folium.Marker([final_df.loc[i]['LATITUDE'], final_df.loc[i]['LONGITUDE']],
                          popup = final_df.loc[i]['ADDRESS'],
                          # icon=folium.Icon(color="red"),
                         ).add_to(hm)
        
# Add mrt locations
tmp = mrt_df.copy()
for i in range(tmp.shape[0]):
    folium.Marker([tmp.loc[i]['LATITUDE'], tmp.loc[i]['LONGITUDE']],
                  popup = tmp.loc[i]['BUILDING'],
                  icon=folium.Icon(color="green"),
                 ).add_to(hm)

hm

### Search Boundaries 
1. This creates a boundary around a specific location

In [10]:
# Create a separation between within radius and outside
# Note that this is an inefficient test of separation

In [11]:
# Identify location
selected_house = final_df.loc[0]
other_houses = final_df.loc[1:]
selected_house_latlong = (float(selected_house['LATITUDE']), float(selected_house['LONGITUDE']))

# This is the start of what my function does
houses_lat = other_houses['LATITUDE'].tolist()
houses_long = other_houses['LONGITUDE'].tolist()
other_houses['dist'] = [geopy.distance.geodesic((float(i), float(j)), selected_house_latlong).m for i,j in zip(houses_lat, houses_long)]
selection_radius = 1000

selected_homes = other_houses[other_houses['dist'] <= selection_radius].reset_index(drop=True)
unselected_homes = other_houses[other_houses['dist'] > selection_radius].reset_index(drop=True)

In [12]:
# Two checks to ensure that my function works! 
print(table_select_from_pt(other_houses, selected_house_latlong).equals(selected_homes))
print(table_select_from_pt(other_houses, selected_house_latlong, select=False).equals(unselected_homes))

True
True


In [66]:
# Create Map with starting point
hm = folium.Map(location = selected_house_latlong, titles="OpenStreetMap", zoom_start=15)

# Houses within selected house
for i in range(selected_homes.shape[0]):
    folium.Marker([selected_homes.loc[i]['LATITUDE'], selected_homes.loc[i]['LONGITUDE']],
                      popup = selected_homes.loc[i]['ADDRESS'],
                      icon=folium.Icon(color="red"),
                     ).add_to(hm)

# Houses outside of selected house
for i in range(unselected_homes.shape[0]):
    folium.Marker([unselected_homes.loc[i]['LATITUDE'], unselected_homes.loc[i]['LONGITUDE']],
                      popup = unselected_homes.loc[i]['ADDRESS'],
                      icon=folium.Icon(color="blue"),
                     ).add_to(hm)

# Add selected marker
folium.Marker([selected_house['LATITUDE'], selected_house['LONGITUDE']],
              popup = selected_house['ADDRESS'],
              icon=folium.Icon(color="black"),
             ).add_to(hm)

# Add selected marker Circle
folium.Circle(
    location=[selected_house['LATITUDE'], selected_house['LONGITUDE']],
    radius=selection_radius,  # radius in meters
    color='blue',
    fill=True,
    fill_color='blue'
).add_to(hm)

In [67]:
table_select_from_pt(mrt_df, selected_house_latlong)

,code_string,code_num,BUILDING,POSTAL,X,Y,LATITUDE,LONGITUDE,dist
0,NS,16,ANG MO KIO,569811,29807.261147,39105.735995,1.369933,103.849558,908.96608


In [68]:
selected_mrt = table_select_from_pt(mrt_df, selected_house_latlong)
selected_mrt

,code_string,code_num,BUILDING,POSTAL,X,Y,LATITUDE,LONGITUDE,dist
0,NS,16,ANG MO KIO,569811,29807.261147,39105.735995,1.369933,103.849558,908.96608


In [69]:
# Select mrts that are near selected homes
tmp = selected_mrt.copy()
for i in range(tmp.shape[0]):
    folium.Marker([tmp.loc[i]['LATITUDE'], tmp.loc[i]['LONGITUDE']],
                  popup = tmp.loc[i]['BUILDING'],
                  icon=folium.Icon(color="green"),
                 ).add_to(hm)

hm

### Routes
#### Making API calls to get routes 
1. Need to make API call for routes
1. Need more ways to make this code easier

In [70]:
target_route = float(unselected_homes.loc[0].LATITUDE), float(unselected_homes.loc[0].LONGITUDE)

# Driving
# route_output = om.make_route_api(selected_house_latlong, target_route)
# total_d = om.get_total_route_dist(route_output)

# Public transport route
# route_output = om.make_route_api(selected_house_latlong, target_route)
# total_d = om.get_total_route_dist(route_output)

# Walking route
route_output = om.make_route_api(selected_house_latlong, target_route, route_type='walk')
total_d = om.get_total_route_dist(route_output)

In [71]:
# Processing walking route
route_g = route_output.json().get('route_geometry')
l = polyline.decode(route_g)
routing = pd.concat([pd.Series(l), pd.Series(l)], axis=1)
routing[1] = routing[1].shift(-1)
route_list = [(i,j) for i,j in zip(routing[0], routing[1]) if j != None]

In [72]:
from folium.plugins import TagFilterButton
hm = folium.Map(location = selected_house_latlong, titles="OpenStreetMap", zoom_start=15)

# Plotting start marker
marker = folium.Marker(location=routing.loc[0][0])
marker.add_to(hm)

# Plotting end marker
marker = folium.Marker(location=routing.loc[routing.shape[0]-1][0])
marker.add_to(hm)

# Plotting route
for stops in route_list:
    line = folium.PolyLine(locations=[stops], 
                           tags=['Homes'], 
                           tooltip=f"Total Distance: {total_d :,.0f}m")
    line.add_to(hm)

# Create map and add the data with additional parameter tags as the segmentation
for i in range(selected_homes.shape[0]):
    folium.Marker([selected_homes.loc[i]['LATITUDE'], selected_homes.loc[i]['LONGITUDE']],
                  popup = selected_homes.loc[i]['ADDRESS'],
                  icon=folium.Icon(color="red"),
                  tags=['Homes']
                 ).add_to(hm)

# Create buttons
categories = ['Homes', "Offices"]
TagFilterButton(categories).add_to(hm)

hm

## Feature design 1
- Put one location, makes API calls, and searches for MRTs around X,XXX metres

In [170]:
# Put location
keyword = "20 Clover Crescent"
select_radius = 1000

# Make OneMap API call
search_location_json = om.loc_best_match(keyword)
search_location_latlong = (search_location_json.get("LATITUDE"), search_location_json.get("LONGITUDE"))

# Do location filter
selected_mrt = table_select_from_pt(mrt_df, search_location_latlong, select_radius=select_radius)

## Create Map
# hm = folium.Map(location = search_location_latlong, titles="OpenStreetMap", zoom_start=15)
# hm = folium.Map(location = search_location_latlong, titles="Stamen Watercolor", zoom_start=15)
hm = folium.Map(location = search_location_latlong, titles="cartodbpositron", zoom_start=15)

# Add selected marker
folium.Marker(search_location_latlong,
              popup = search_location_json.get('ADDRESS'),
              tooltip = search_location_json.get('ADDRESS'),
              icon=plugins.BeautifyIcon(
                  icon="house",
                  icon_shape="circle",
                  text_color="white",
                  background_color='black',
                  border_color="black"),
             ).add_to(hm)

# Add selected marker Circle
folium.Circle(
    location=search_location_latlong,
    radius=select_radius,  # radius in meters
    color='blue',
    fill=True,
    fill_color='blue'
).add_to(hm)

group1 = folium.FeatureGroup(name='MRTs')

# MRTs within range of target home
tmp = selected_mrt.copy()
for i in range(tmp.shape[0]):
    
    # Set important marker & route info
    marker_info, marker_lat, marker_long = tmp.loc[i]['BUILDING'], tmp.loc[i]['LATITUDE'], tmp.loc[i]['LONGITUDE']

    # Add walking routes to MRT
    route_color = generate_color()
    route, total_d = create_route(search_location_latlong, (marker_lat, marker_long))
    tooltip_msg = f"{search_location_json.get('ADDRESS')} to {tmp.loc[i].BUILDING} <br>Total Distance: {total_d :,.0f}m"
    for stops in route:
        line = folium.PolyLine(locations=[stops],
                               color=route_color,
                               weight=5,
                               tooltip=tooltip_msg)
        line.add_to(group1)

    # MRT markers
    folium.Marker([marker_lat, marker_long],
                  popup = marker_info,
                  tooltip = tooltip_msg,
                  icon=plugins.BeautifyIcon(
                      icon="train-subway",
                      text_color="white",
                      icon_shape="circle",
                      background_color=route_color,
                      border_color=route_color),
                 ).add_to(group1)

group1.add_to(hm)

# Add layer control to switch between groups
folium.LayerControl().add_to(hm)

# Display the map
hm

#### Create HTMLs from folium 

In [177]:
filename = 'Test MRT Routes'
title_html = f'<h1 style="position:absolute;z-index:100000;left:40vw" >{filename}</h1>'
hm.get_root().html.add_child(folium.Element(title_html))
hm.save(f'{filename}.html')
time.sleep(1)

### SpatialQuery
1. Need to see if I can create spatial queries and show them on folium
1. Makes sense that this requires backend processing!

### Regions 
Need to see if I can create regional overlays onto Folium maps

### Back Up Code

In [146]:
# Keep this code for future reference, but adding HTML tags to Folium maps seems rather not intuitive
# HTML template for the table
table_rows = ""
for (lat, lon, val) in zip(tmp['LATITUDE'].tolist(), tmp['LONGITUDE'].tolist(), tmp['BUILDING'].tolist()):
    table_rows += f"""
        <tr>
            <td style="border: 1px solid black; padding: 5px;">{lat:.4f}, {lon:.4f}</td>
            <td style="border: 1px solid black; padding: 5px;">{val}</td>
        </tr>
    """

html = f"""
<div style="position: fixed; bottom: 50px; right: 50px; width: 300px; height: auto; 
     background-color: white; border:2px solid grey; z-index:9999; font-size:14px;">
    <h4 style="margin-top: 10px; text-align:center;">Calculated Values</h4>
    <table style="width: 100%; border-collapse: collapse; margin: 10px;">
        <tr>
            <th style="border: 1px solid black; padding: 5px;">Location</th>
            <th style="border: 1px solid black; padding: 5px;">Value</th>
        </tr>
        {table_rows}
    </table>
</div>
"""

# # Add HTML element to the map
# from folium import IFrame
# iframe = IFrame(html, width=430, height=300)
# popup = folium.Popup(iframe, max_width=350)
folium.Marker(selected_house_latlong, icon=folium.DivIcon(
    icon_size=(100,20),
    icon_anchor=(5,5),
    html=f"""<div style="display:none;"></div>"""), 
              popup=popup).add_to(hm)

folium.map.Marker(location=search_location_latlong,
                  icon=folium.DivIcon(
                      icon_size=(100,20),
                      icon_anchor=(5,5),
                      html=html)
                 ).add_to(hm)

1.31933570922513 103.861569454156 BOON KENG
1.33137949021367 103.869055656782 POTONG PASIR
1.3213011022832 103.871622627142 GEYLANG BAHRU
